# VGG网络
- 将卷积层、激活函数、池化层放在一个模块（函数）中
- 这样显得整个网络更加清晰

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
# 定义vgg网络
# 参数分别为：卷积层数、输入通道数，输出通道数
def vgg_block(num_convs, input_channel, output_channel):
    # 最后这个模块输出的神经网络
    blk = []
    for i in range(num_convs):
        # 卷积层
        blk.append(nn.Conv2d(input_channel, output_channel, 
        kernel_size=3, padding=1))
        # 激活函数
        blk.append(nn.ReLU()) 
        # 后面一层的输入是当前层的输出
        input_channel = output_channel
    # 最后再链接一个最大池化层
    blk.append(nn.MaxPool2d(kernel_size=2, stride=2))
    # 返回这个模块
    return nn.Sequential(*blk)

In [4]:
# 定义vgg网络
# 两个参数分别表示：卷积层数、输出通道数
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [9]:
# 定义vgg网络
def vgg(conv_arch):
    conv_blocks = []
    # 初始化第一层的输入通道数
    in_channel = 1
    for(num_convs, out_chanels) in conv_arch:
        conv_blocks.append(vgg_block(num_convs, in_channel, out_chanels))
        in_channel = out_chanels
    return nn.Sequential(
        *conv_blocks,
        nn.Flatten(),
        nn.Linear(out_chanels*7*7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10)
    )
net = vgg(conv_arch)

In [11]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, 'output shape: ', X.shape)

Sequential output shape:  torch.Size([1, 64, 112, 112])
Sequential output shape:  torch.Size([1, 128, 56, 56])
Sequential output shape:  torch.Size([1, 256, 28, 28])
Sequential output shape:  torch.Size([1, 512, 14, 14])
Sequential output shape:  torch.Size([1, 512, 7, 7])
Flatten output shape:  torch.Size([1, 25088])
Linear output shape:  torch.Size([1, 4096])
ReLU output shape:  torch.Size([1, 4096])
Dropout output shape:  torch.Size([1, 4096])
Linear output shape:  torch.Size([1, 4096])
ReLU output shape:  torch.Size([1, 4096])
Dropout output shape:  torch.Size([1, 4096])
Linear output shape:  torch.Size([1, 10])


In [12]:
ratio = 4
small_conv_arch = [(pair[0], pair[1]//ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [ ]:
lr, num_epochs, batch_size = 0.05, 5, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())